Для работы с Google Sheets сначала необходимо пройти процедуру авторизации с помощью Google Developers Console.

Она аналогична той, что мы делали в прошлом модуле для API Google Analytics.

Итак, заходим в консоль разработчика https://console.developers.google.com. В ней должен остаться доступ к Google Analytics Reporting API с прошлого модуля (но в данном модуле он нам не пригодится). Нажимаем на "Включить API и сервисы":


In [6]:
import httplib2
import argparse
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

Задаем основные параметры приложения, от имени которого будем делать запросы к Google Sheets API:

In [7]:
# Разрешения на просмотр и редактирование
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
# Название файла с идентификаторами
CLIENT_SECRET_FILE = 'client_secret_grants.json'
# Название приложения для нашего онлайн-отчета (можно использовать любое)
APPLICATION_NAME = 'Google Sheets API Report'
# название файла, в который после процедуры авторизации будут записаны токены (сейчас его еще не должно быть в папке)
# для Google Analytics этот файл назывался 'analytics.dat', но в разработчики Google Sheets API выбрали другое название
# чтобы вам при необходимости было проще ориентироваться в документации Google API, мы сохранили такую же систему обозначений
credential_path = 'sheets.googleapis.com-report.json'

Проходим процесс авторизации для работы с Google Sheets в первый раз:

В открывшемся окне выбираем нужный аккаунт (если у вас их несколько).
Даем вашему приложению разрешение на "Просмотр и изменение электронных таблиц на Google Диске".
Процесс должен завершиться со служебным сообщением "The authentication flow has completed".

In [10]:
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
    print('Storing credentials to ' + credential_path)

В вашей папке должен появиться файл "sheets.googleapis.com-report.json", в котором содержится токен access_token для запросов к API Google Sheets и refresh_token для его обновления.

Формируем объект service, с помощью которого мы будем читать и записывать данные Google Sheets:

In [11]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)

Отлично, теперь мы полностью готовы к автоматизированной работе с Google Sheets.

На этом шаге мы научимся считывать данные из Google-таблиц.

Этот алгоритм может сильно упростить вам многие задачи обработки данных, т. к. вы и ваши коллеги можете обновлять необходимые словари, зависимости и другие таблицы онлайн. К тому же в Google Sheets можно распределять доступы на чтение и редактирование таблиц.

Давайте разберем это на примере ведения отчетности маркетинговых кампаний.

Допустим, на ваш сайт приходит трафик от нескольких партнеров (назовем их СМИ2, РБК и Infox). У каждого партнера в ссылках на ваш сайт имеется свой набор UTM-меток.

Справочная информация: под UTM-меткой подразумевается принятая в маркетинге система обозначений параметров ссылок, которые автоматически распознаются многими системами веб-аналитики как название источника и канала. Пример ссылки с UTM-метками:

https://my-site.ru/?utm_source=infox&utm_medium=cpc

Вам необходимо составить отчет по количеству визитов, продаж и прочим метрикам в разбивке по партнерам СМИ2, РБК и Infox.

Проблема в том, что названия UTM-меток могут меняться (обычно так и случается в реальной жизни). Конечно, можно договориться о том, что вам при любом обновлении будут присылать обновленный список названий UTM-меток. Скорее всего, работать это не будет.

Более простым вариантом является ведение онлайн-таблицы, в которой будут отмечаться эти изменения.

Обычно такие таблицы в том или ином виде ведутся (иногда, правда, в Excel). Перенести их в Google Sheets обычно не составляет труда.

В итоге, если вам составят обновляемую таблицу с UTM-метками, то вы (и ваш скрипт) всегда сможете узнать, какой набор UTM-меток какому партнеру соответствует:

Для того, чтобы автоматически прочитать ее содержимое, нам понадобятся два параметра:

1. ID книги Google Sheets. Для ее получения скопируйте адрес страницы, на которой расположена ваша таблица словаря меток.

URL должен иметь вид https://docs.google.com/spreadsheets/d/136LYwwrKSQ2bct8h29_yjeXo-70RTwA_-VGF5KONrIM/edit#gid=0
2. Для чтения данных нам также необходимо указать название листа (если их несколько) и диапазон.

Например, в данном модуле у нас будет 2 листа: текущий можно назвать "partners dict", а также мы сделаем онлайн-отчет на листе "daily_metrics".

Сама таблица помещается в диапазон A1:C5. Поскольку этот список в будущем может вырасти, то возьмем с запасом до строки 10:
spreadsheetId = '136LYwwrKSQ2bct8h29_yjeXo-70RTwA_-VGF5KONrIM'

In [12]:
spreadsheetId = '136LYwwrKSQ2bct8h29_yjeXo-70RTwA_-VGF5KONrIM'
rangeName = 'partners dict!A1:C10'

Все готово, делаем запрос к API. Результат записываем в переменную values:

In [13]:
result = service.spreadsheets().values().get(spreadsheetId = spreadsheetId, range = rangeName).execute()
values = result.get('values', [])
print(values)

[['Партнер', 'utm_source', 'utm_medium'], ['СМИ2', 'smi2', 'cpm'], ['РБК', 'rbc', 'cpc'], ['Infox', 'infox', 'cpc'], ['Infox', 'infox-action', 'cpa']]


Запрос автоматически убрал из ответа пустые строки. Теперь лист values можно использовать как словарь для вашей отчетности по партнерам.

Подготовимся к импорту данных из Яндекс.Метрики: в файл config.yaml пишем токен для запросов к API. В файле params.yaml в строку с spreadsheet_id необходимо вставить ID вашей книги. В файле report_params.yaml записываются параметры отчетов, которые мы хотим визуализировать:

Файл report_params.yaml

dau — служебное название отчета. Если вы захотите добавить еще один отчет, то сделайте копию этого названия со всем параметрами под ним.
dimensions и metrics — измерения и метрики, которые будем отправлять в запросе к API Яндекс.Метрики.
sort — метрика, по которой будем сортировать отчет. В нашем случае это дата отчета.
days_ago — за какой период выгружаем отчет. В нашем варианте - за последние 14 дней.
spreadsheet_id — ID книги в Google Sheets, где расположен отчет (подставьте ID своей книги).
range_name — название листа отчета (daily_metrics), диапазон столбцов, в которых будет располагаться выгрузка (с A по E).
Далее импортируем те же методы и функции, которые использовали при работе с API Яндекс.Метрики:

In [14]:
from yaml import load
from datetime import datetime, timedelta
import requests
from pprint import pprint

In [15]:
token = 'AQAAAAATeulOAAWPR1HqyVDuRET3vmW66cppHDU'

In [17]:
f = open('report_params.yaml', 'r')
params = load(f)
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2018-02-25'
endDate = '2018-03-10'
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

Используем функции для запроса и обработки ответа API из прошлых модулей:

In [18]:
def make_request(**kwargs):
    """Запрос к API Яндекс.Метрики. Возвращает JSON ответа"""   
    return requests.get(API_URL, params = kwargs).json()
def json_handler(data):
    """Преобразование ответа из JSON в список листов"""
    report = []
    for line in data['data']:
        dimensions_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append( dimensions_list + metrics_list )    
    return report

In [19]:
response = make_request(date1 = startDate, date2 = endDate, dimensions = dimensions, metrics = metrics, 
                    id = counter, sort = sortby, oauth_token = token, accuracy = 1)

In [20]:
data = json_handler(response)
data

[['2018-02-25', 202.0, 193.0, 386.0, 15.34653465],
 ['2018-02-26', 279.0, 264.0, 440.0, 13.62007168],
 ['2018-02-27', 280.0, 258.0, 481.0, 18.21428571],
 ['2018-02-28', 294.0, 278.0, 399.0, 15.98639456],
 ['2018-03-01', 293.0, 272.0, 430.0, 15.35836177],
 ['2018-03-02', 317.0, 284.0, 441.0, 12.30283912],
 ['2018-03-03', 342.0, 292.0, 551.0, 15.78947368],
 ['2018-03-04', 265.0, 233.0, 429.0, 19.62264151],
 ['2018-03-05', 282.0, 257.0, 427.0, 17.0212766],
 ['2018-03-06', 258.0, 229.0, 389.0, 17.82945736],
 ['2018-03-07', 203.0, 185.0, 318.0, 12.31527094],
 ['2018-03-08', 198.0, 168.0, 382.0, 11.61616162],
 ['2018-03-09', 258.0, 232.0, 514.0, 18.21705426],
 ['2018-03-10', 221.0, 198.0, 429.0, 20.36199095]]

In [21]:
import pandas as pd
labels = ['date', 'visits', 'users', 'views','cancels']
df = pd.DataFrame.from_records(data, columns=labels)
df['visits'].sum()

3692.0

Данные получили, осталось отправить их в Google Sheets. Сначала сформируем название диапазона (daily_metrics!A2:E15), в который отправим данные Метрики:

In [22]:
range_data = params['reports']['dau']['range_name']
range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)


In [23]:
print(range_name)

daily_metrics!A2:E15


Для отправки листа data в Google Sheets его необходимо преобразовать в словарь:

In [24]:
body = {'values': data}
pprint(body)

{'values': [['2018-02-25', 202.0, 193.0, 386.0, 15.34653465],
            ['2018-02-26', 279.0, 264.0, 440.0, 13.62007168],
            ['2018-02-27', 280.0, 258.0, 481.0, 18.21428571],
            ['2018-02-28', 294.0, 278.0, 399.0, 15.98639456],
            ['2018-03-01', 293.0, 272.0, 430.0, 15.35836177],
            ['2018-03-02', 317.0, 284.0, 441.0, 12.30283912],
            ['2018-03-03', 342.0, 292.0, 551.0, 15.78947368],
            ['2018-03-04', 265.0, 233.0, 429.0, 19.62264151],
            ['2018-03-05', 282.0, 257.0, 427.0, 17.0212766],
            ['2018-03-06', 258.0, 229.0, 389.0, 17.82945736],
            ['2018-03-07', 203.0, 185.0, 318.0, 12.31527094],
            ['2018-03-08', 198.0, 168.0, 382.0, 11.61616162],
            ['2018-03-09', 258.0, 232.0, 514.0, 18.21705426],
            ['2018-03-10', 221.0, 198.0, 429.0, 20.36199095]]}


Наконец, отправляем эти данные в Google Sheets. Опция valueInputOption дает команду API на то, что значения надо вставить так, как если бы пользователь ввел их вручную в веб-интерфейсе. Подробнее: https://developers.google.com/sheets/api/reference/rest/v4/ValueInputOption

In [25]:
service.spreadsheets().values().update( spreadsheetId = spreadsheetId, range = range_name, valueInputOption = 'USER_ENTERED', body = body ).execute()

{'spreadsheetId': '136LYwwrKSQ2bct8h29_yjeXo-70RTwA_-VGF5KONrIM',
 'updatedCells': 70,
 'updatedColumns': 5,
 'updatedRange': 'daily_metrics!A2:E15',
 'updatedRows': 14}

В этом шаге мы познакомимся с основами простых визуализаций в Google Sheets – добавим вишенку на торт вашего отчета.

Для быстрой отрисовки графика выделите диапазон A1:B15. В верхнем меню во вкладке "Вставка" выберите "Диаграмма":